In [1]:
!git clone https://github.com/gauthiercaceres/bike_counters.git

Cloning into 'bike_counters'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 107 (delta 35), reused 36 (delta 34), pack-reused 57 (from 1)
Receiving objects: 100% (107/107), 1.44 MiB | 5.96 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd bike_counters

/content/bike_counters


importing libraries

In [70]:
# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Advanced ensemble methods
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    StackingRegressor,
    ExtraTreesRegressor
)
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import gc  # Garbage collection
import importlib

warnings.filterwarnings('ignore')

# Data handling and ML libraries
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    StackingRegressor,
    ExtraTreesRegressor
)
import external_data.example_estimator  # For external data processing
importlib.reload(external_data.example_estimator)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, date
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_log_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)

merging with ext data

In [75]:
data_path='data/train.parquet'
test_path='data/final_test.parquet'
train_data = pd.read_parquet(Path(data_path))
print(train_data.columns)
test_data = pd.read_parquet(Path(test_path))



Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count'],
      dtype='object')


In [76]:
train_data = external_data.example_estimator._encode_dates(train_data)

In [73]:
train_data.head(4)

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,year,month,day,weekday,hour,holiday,week_of_month,workday,season
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,2,0,1,1,3
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,2020,9,1,1,3,0,1,1,3
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,1,4,0,1,1,3
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,2020,9,1,1,15,0,1,1,3


In [77]:
train_data = external_data.example_estimator._merge_external_data(train_data)


In [78]:
train_data.columns

Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'bike_count',
       'date', 'counter_installation_date', 'coordinates',
       'counter_technical_id', 'latitude', 'longitude', 'log_bike_count',
       'year', 'month', 'day', 'weekday', 'hour', 'holiday', 'week_of_month',
       'workday', 'season', 'numer_sta', 'pmer', 'tend', 'cod_tend', 'dd',
       'ff', 't', 'td', 'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas', 'cl',
       'cm', 'ch', 'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12',
       'tx24', 'tminsol', 'sw', 'tw', 'raf10', 'rafper', 'per', 'etat_sol',
       'ht_neige', 'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24',
       'phenspe1', 'phenspe2', 'phenspe3', 'phenspe4', 'nnuage1', 'ctype1',
       'hnuage1', 'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3',
       'hnuage3', 'nnuage4', 'ctype4', 'hnuage4'],
      dtype='object')

In [80]:
df =train_data

In [81]:
import pandas as pd

# Feature Engineering Example
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['day_period'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])
df['installation_age_days'] = (pd.to_datetime(df['date']) - pd.to_datetime(df['counter_installation_date'])).dt.days

# Dropping Less Relevant Features
df = df.drop(columns=['counter_name', 'site_name', 'week_of_month'])
from geopy.distance import distance

# Coordinates of Paris city center
city_center = (48.8566, 2.3522)

# Add a new column 'distance_to_center' to the DataFrame
df['distance_to_center'] = df.apply(
    lambda row: distance(city_center, (row['latitude'], row['longitude'])).km,
    axis=1
)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
df['location_type'] = kmeans.fit_predict(df[['latitude', 'longitude']])
df['is_rainy'] = (df['rr1'] > 0).astype(int)
df['is_windy'] = (df['ff'] > 8).astype(int)
df['low_visibility'] = (df['vv'] < 1000).astype(int)
df['total_precipitation_24h'] = df[['rr1', 'rr3', 'rr6', 'rr12', 'rr24']].sum(axis=1)

df['temperature_c'] = df['t'] - 273.15  # Kelvin to Celsius conversion
df['is_hot'] = (df['temperature_c'] > 20).astype(int)  # Indicates hot weather
df['is_cold'] = (df['temperature_c'] < 5).astype(int)   # Indicates cold weather
df['is_snowy'] = (df['temperature_c'] < 0).astype(int)
df['bad_weather'] = ((df['is_rainy'] == 1) | (df['is_snowy'] == 1) | (df['low_visibility'] == 1)).astype(int)




In [95]:
selected_features = [
    # Key Date and Time Features
    'year', 'month', 'day', 'weekday', 'hour', 'holiday', 'workday', 'season',
    'is_weekend', 'day_period', 'installation_age_days',

    # Geospatial Information
    'latitude', 'longitude', 'coordinates', 'distance_to_center', 'location_type',

    # Weather-Related Features
    'temperature_c', 'is_hot', 'is_cold', 'is_rainy', 'is_windy', 'low_visibility',
    'total_precipitation_24h', 'bad_weather'
]


In [96]:
final_train_data = df.loc[:, selected_features + ['log_bike_count', 'bike_count']]


In [97]:
final_train_data.columns

Index(['year', 'month', 'day', 'weekday', 'hour', 'holiday', 'workday',
       'season', 'is_weekend', 'day_period', 'installation_age_days',
       'latitude', 'longitude', 'coordinates', 'distance_to_center',
       'location_type', 'temperature_c', 'is_hot', 'is_cold', 'is_rainy',
       'is_windy', 'low_visibility', 'total_precipitation_24h', 'bad_weather',
       'log_bike_count', 'bike_count'],
      dtype='object')

handling missing vals

In [105]:
# Count the number of missing values in each column
missing_values_train = pd.DataFrame(final_train_data.isna().sum(), columns=["missing_count"])

# Filter columns where missing values are greater than 0
columns_with_missing_values = missing_values_train[missing_values_train["missing_count"] > 0]

# Get the count of columns with missing values
num_columns_with_missing_values = columns_with_missing_values.shape[0]

# Display the results
print(f"Number of columns with missing values: {num_columns_with_missing_values}")
print(columns_with_missing_values)



Number of columns with missing values: 1
            missing_count
day_period          20598


dropping cols with >40% missing vals

In [106]:
# Calculate the percentage of missing values for each column
missing_percentage = final_train_data.isna().mean() * 100

# Identify columns with more than 40% missing values
cols_to_drop = missing_percentage[missing_percentage > 40].index

# Drop those columns from the DataFrame
train_data_cleaned = final_train_data.drop(columns=cols_to_drop)

# Check the shape of the updated DataFrame
print(f"Columns dropped: {cols_to_drop}")
print(f"Updated DataFrame shape: {train_data_cleaned.shape}")

# Check the remaining missing values
print("Missing values after dropping columns with > 40% NaN values:")
print(train_data_cleaned.isna().sum())


Columns dropped: Index([], dtype='object')
Updated DataFrame shape: (496827, 26)
Missing values after dropping columns with > 40% NaN values:
year                           0
month                          0
day                            0
weekday                        0
hour                           0
holiday                        0
workday                        0
season                         0
is_weekend                     0
day_period                 20598
installation_age_days          0
latitude                       0
longitude                      0
coordinates                    0
distance_to_center             0
location_type                  0
temperature_c                  0
is_hot                         0
is_cold                        0
is_rainy                       0
is_windy                       0
low_visibility                 0
total_precipitation_24h        0
bad_weather                    0
log_bike_count                 0
bike_count                     0


In [107]:
import numpy as np
import pandas as pd

# Function to handle missing values and infinities
def handle_missing_and_infs(df):
    for column in df.columns:
        if df[column].dtype in ['float64', 'int64']:  # Numeric columns only
            # Replace inf/-inf with NaN
            df[column].replace([np.inf, -np.inf], np.nan, inplace=True)
            # Fill NaN values with the mean
            df[column].fillna(df[column].mean(), inplace=True)
        else:  # Non-numeric columns
            if df[column].isna().sum() > 0:  # Handle NaN for non-numeric
                if df[column].dtype == 'object':
                    df[column].fillna(df[column].mode()[0], inplace=True)  # Most frequent value
                elif pd.api.types.is_categorical_dtype(df[column]):
                    df[column].fillna(df[column].mode()[0], inplace=True)  # Mode for categorical
    return df

# Apply the function to the train_data
train_data_cleaned = handle_missing_and_infs(train_data_cleaned)

# Check if missing values and infinities are handled
print("Missing values after handling:")
print(train_data_cleaned.isna().sum())


Missing values after handling:
year                       0
month                      0
day                        0
weekday                    0
hour                       0
holiday                    0
workday                    0
season                     0
is_weekend                 0
day_period                 0
installation_age_days      0
latitude                   0
longitude                  0
coordinates                0
distance_to_center         0
location_type              0
temperature_c              0
is_hot                     0
is_cold                    0
is_rainy                   0
is_windy                   0
low_visibility             0
total_precipitation_24h    0
bad_weather                0
log_bike_count             0
bike_count                 0
dtype: int64


In [108]:
# Count the number of missing values in each column
missing_values_train = pd.DataFrame(train_data_cleaned.isna().sum(), columns=["missing_count"])

# Filter columns where missing values are greater than 0
columns_with_missing_values = missing_values_train[missing_values_train["missing_count"] > 0]

# Get the count of columns with missing values
num_columns_with_missing_values = columns_with_missing_values.shape[0]

# Display the results
print(f"Number of columns with missing values: {num_columns_with_missing_values}")
print(columns_with_missing_values)


Number of columns with missing values: 0
Empty DataFrame
Columns: [missing_count]
Index: []


encoding using label encoder

In [109]:
from sklearn.preprocessing import LabelEncoder
import pickle  # To save encoders

# Dictionary to save encoders for all categorical columns
label_encoders = {}

# Identify categorical columns
categorical_cols = train_data_cleaned.select_dtypes(include=['object', 'category']).columns

# Process each categorical column
for col in categorical_cols:
    unique_count = train_data_cleaned[col].nunique()

    if unique_count > 50:
        print(f"Dropping column '{col}' with {unique_count} unique categories.")
        train_data_cleaned.drop(columns=[col], inplace=True)
    else:
        print(f"Encoding column '{col}' with {unique_count} unique categories.")
        # Initialize label encoder
        le = LabelEncoder()
        # Fit and transform the column
        train_data_cleaned[col] = le.fit_transform(train_data_cleaned[col].astype(str))
        # Save the label encoder for future use
        label_encoders[col] = le

# Save the encoders dictionary to a file
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

# Check the updated dataset
print("Updated dataset:")
print(train_data_cleaned.head())


Encoding column 'day_period' with 4 unique categories.
Encoding column 'coordinates' with 30 unique categories.
Updated dataset:
     year  month  day  weekday  hour  holiday  workday  season  is_weekend  \
107  2020      9    1        1     2        0        1       3           0   
153  2020      9    1        1     3        0        1       3           0   
201  2020      9    1        1     4        0        1       3           0   
782  2020      9    1        1    15        0        1       3           0   
925  2020      9    1        1    18        0        1       3           0   

     day_period  installation_age_days   latitude  longitude  coordinates  \
107           3                   2783  48.846028   2.375429           10   
153           3                   2783  48.846028   2.375429           10   
201           3                   2783  48.846028   2.375429           10   
782           0                   2783  48.846028   2.375429           10   
925           0  

In [110]:
# Count the number of missing values in each column
missing_values_train = pd.DataFrame(train_data_cleaned.isna().sum(), columns=["missing_count"])

# Filter columns where missing values are greater than 0
columns_with_missing_values = missing_values_train[missing_values_train["missing_count"] > 0]

# Get the count of columns with missing values
num_columns_with_missing_values = columns_with_missing_values.shape[0]

# Display the results
print(f"Number of columns with missing values: {num_columns_with_missing_values}")
print(columns_with_missing_values)

Number of columns with missing values: 0
Empty DataFrame
Columns: [missing_count]
Index: []


In [111]:
train_data_cleaned.columns

Index(['year', 'month', 'day', 'weekday', 'hour', 'holiday', 'workday',
       'season', 'is_weekend', 'day_period', 'installation_age_days',
       'latitude', 'longitude', 'coordinates', 'distance_to_center',
       'location_type', 'temperature_c', 'is_hot', 'is_cold', 'is_rainy',
       'is_windy', 'low_visibility', 'total_precipitation_24h', 'bad_weather',
       'log_bike_count', 'bike_count'],
      dtype='object')

outlier detection and capping

In [112]:

def cap_outliers(df, threshold=5):
    # Iterate over all numeric columns
    for col in df.select_dtypes(include=[np.number]).columns:
        # Calculate the mean and standard deviation for the column
        mean = df[col].mean()
        std_dev = df[col].std()

        # Define the lower and upper bounds
        lower_limit = mean - threshold * std_dev
        upper_limit = mean + threshold * std_dev

        # Cap the outliers
        df[col] = np.clip(df[col], lower_limit, upper_limit)

    return df

# Apply the outlier capping function
train_data_cleaned = cap_outliers(train_data_cleaned, threshold=3)

# Display the cleaned DataFrame
print(train_data_cleaned)


        year  month  day  weekday  hour  holiday  workday  season  is_weekend  \
107     2020      9    1        1     2        0        1       3           0   
153     2020      9    1        1     3        0        1       3           0   
201     2020      9    1        1     4        0        1       3           0   
782     2020      9    1        1    15        0        1       3           0   
925     2020      9    1        1    18        0        1       3           0   
...      ...    ...  ...      ...   ...      ...      ...     ...         ...   
495819  2021      9    9        3     6        0        1       3           0   
496091  2021      9    9        3    10        0        1       3           0   
496354  2021      9    9        3    15        0        1       3           0   
496719  2021      9    9        3    22        0        1       3           0   
496826  2021      9    9        3    23        0        1       3           0   

        day_period  install

scaling

In [113]:
# from sklearn.preprocessing import StandardScaler

# # Identify numeric columns
# numeric_cols = train_data_cleaned.select_dtypes(include=['float64', 'int64']).columns

# # Exclude 'bike_count' and 'log_bike_count' from scaling
# columns_to_scale = numeric_cols.difference(['bike_count', 'log_bike_count'])

# # Initialize the scaler
# scaler = StandardScaler()

# # Scale only the selected numeric columns
# scaled_data = scaler.fit_transform(train_data_cleaned[columns_to_scale])

# # Create a new DataFrame with only the scaled numeric columns
# df_train_scaled = pd.DataFrame(scaled_data, columns=columns_to_scale)

# # Check the scaled data
# print("Scaled dataset:")
# print(df_train_scaled.head())


Scaled dataset:
   bad_weather  coordinates  day_period  distance_to_center  holiday  \
0      -0.3894    -0.480358    1.335842           -0.628134      0.0   
1      -0.3894    -0.480358    1.335842           -0.628134      0.0   
2      -0.3894    -0.480358    1.335842           -0.628134      0.0   
3      -0.3894    -0.480358   -1.264111           -0.628134      0.0   
4      -0.3894    -0.480358   -1.264111           -0.628134      0.0   

   installation_age_days   is_cold    is_hot  is_rainy  is_weekend  is_windy  \
0               3.659015 -0.425422 -0.417222  -0.32525     -0.6288 -0.157042   
1               3.659015 -0.425422 -0.417222  -0.32525     -0.6288 -0.157042   
2               3.659015 -0.425422 -0.417222  -0.32525     -0.6288 -0.157042   
3               3.659015 -0.425422  2.396807  -0.32525     -0.6288 -0.157042   
4               3.659015 -0.425422 -0.417222  -0.32525     -0.6288 -0.157042   

   latitude  longitude  low_visibility  temperature_c  \
0 -0.448167  

In [114]:
# # Check for NaN values in scaled_data
# nan_count = np.isnan(scaled_data).sum()

# if nan_count > 0:
#     print(f"Scaled data contains {nan_count} NaN values.")
# else:
#     print("Scaled data has no NaN values.")


Scaled data has no NaN values.


checking for top 15 correlated feats

In [67]:
# df_train_scaled.columns

Index(['cl', 'cod_tend', 'ctype1', 'dd', 'etat_sol', 'ff', 'hbas', 'hnuage1',
       'holiday', 'ht_neige', 'latitude', 'longitude', 'n', 'nbas', 'nnuage1',
       'numer_sta', 'per', 'perssfrai', 'pmer', 'pres', 'raf10', 'rafper',
       'rr1', 'rr12', 'rr24', 'rr3', 'rr6', 'site_id', 'ssfrai', 't', 'td',
       'tend', 'tend24', 'u', 'vv', 'w1', 'w2', 'workday', 'ww'],
      dtype='object')

In [69]:
# df_train_scaled_df = pd.DataFrame(df_train_scaled, columns=columns_to_scale)
# df_train_scaled_df['log_bike_count']=train_data_cleaned['log_bike_count']
# # Calculate the correlation matrix
# correlation_matrix = df_train_scaled_df.corr()


# # Extract the correlations of all columns with the target 'log_bike_count'
# target_correlation = correlation_matrix['log_bike_count']

# # Sort the correlations by their absolute values in descending order
# sorted_corr = target_correlation.abs().sort_values(ascending=False)

# # Get the top 20 columns (excluding the target column itself)
# top_16_features = sorted_corr.index[1:30]  # Skipping the first one because it's the target itself
# # Assuming `df_train_scaled` is a NumPy array and `column_names` is the list of feature names
# df_train_scaled_df = pd.DataFrame(df_train_scaled, columns=columns_to_scale)

# # Now, select the top 16 features by their names
# df_train_scaled_top_features = df_train_scaled_df[top_16_features]


# # Display the selected top features
# print(top_16_features)

Index(['site_id', 'latitude', 'td', 't', 'etat_sol', 'pres', 'pmer',
       'longitude', 'ff', 'raf10', 'nnuage1', 'ww', 'rr12', 'rafper',
       'ht_neige', 'rr24', 'rr1', 'rr6', 'dd', 'vv', 'rr3', 'tend24', 'u',
       'perssfrai', 'tend', 'cod_tend', 'w1', 'ctype1', 'w2'],
      dtype='object')


In [65]:
# df_train_scaled_top_features.columns

Index(['site_id', 'latitude', 'td', 't', 'etat_sol', 'pres', 'pmer',
       'longitude', 'ff', 'raf10', 'nnuage1', 'ww', 'rr12', 'rafper',
       'ht_neige', 'rr24', 'rr1', 'rr6', 'dd', 'vv', 'rr3', 'tend24', 'u',
       'perssfrai'],
      dtype='object')

pca

In [18]:
# from sklearn.decomposition import PCA
# import pandas as pd

# # Apply PCA to the scaled data
# pca_model = PCA()
# pca_model.fit(df_train_scaled)

# # Create columns for the PCA components
# pca_columns = ['PC' + str(i + 1) for i in range(df_train_scaled.shape[1])]

# # Transform the scaled data using PCA
# df_pca_train = pd.DataFrame(pca_model.transform(df_train_scaled), columns=pca_columns)

# # Get explained variance ratio
# explained_info_train = pd.DataFrame(pca_model.explained_variance_ratio_, columns=['Explained Info'])

# # Calculate cumulative explained variance
# explained_info_train['Cumulative Explained Info'] = explained_info_train['Explained Info'].cumsum()

# # Display cumulative explained variance and filter features with cumulative importance > 50%
# explained_info_train_sum = explained_info_train[explained_info_train['Cumulative Explained Info'] <= 0.7]

# # Get the names of features with cumulative importance > 50%
# selected_pcs = explained_info_train_sum.index.tolist()

# # Print the selected principal components
# print(f"Selected principal components (with cumulative importance > 50%): {selected_pcs}")


Selected principal components (with cumulative importance > 50%): [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [19]:
# from sklearn.decomposition import PCA
# import pandas as pd

# # Define the number of components to keep
# n_components = 9

# # Create column names for the PCA components
# pca_columns = ['PC' + str(i + 1) for i in range(n_components)]

# # Apply PCA with the specified number of components
# pca_model = PCA(n_components=n_components)
# pca_model.fit(df_train_scaled)

# # Transform the scaled data to the selected components
# df_pca_train = pd.DataFrame(pca_model.transform(df_train_scaled), columns=pca_columns)

# # Print the resulting DataFrame with the first few rows
# print("DataFrame with 10 PCA components:")
# print(df_pca_train.head())


DataFrame with 10 PCA components:
        PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0 -2.728619 -1.086594  0.345809  1.347857 -0.145793  0.398638 -0.907141   
1 -2.732989 -1.722116  0.758959  1.226226 -0.313448  0.402504 -1.249807   
2 -2.732989 -1.722116  0.758959  1.226226 -0.313448  0.402504 -1.249807   
3 -0.597296  1.981913 -0.950825  1.119922 -0.889010  0.855845 -0.326463   
4 -0.363953  1.531070  0.139376  1.364104 -0.359247  0.269620  0.374853   

        PC8       PC9  
0  0.343817 -0.369767  
1  0.350691 -0.190392  
2  0.350691 -0.190392  
3  0.313637 -0.212039  
4  0.294311 -0.221230  


In [119]:
# train_data_cleaned.columns

Index(['year', 'month', 'day', 'weekday', 'hour', 'holiday', 'workday',
       'season', 'is_weekend', 'day_period', 'installation_age_days',
       'latitude', 'longitude', 'coordinates', 'distance_to_center',
       'location_type', 'temperature_c', 'is_hot', 'is_cold', 'is_rainy',
       'is_windy', 'low_visibility', 'total_precipitation_24h', 'bad_weather',
       'log_bike_count', 'bike_count'],
      dtype='object')

In [120]:
# Drop columns that are not relevant for your analysis or model training
cols_to_drop = ['log_bike_count', 'bike_count']



X = train_data_cleaned.drop(columns=cols_to_drop)
y = train_data_cleaned['log_bike_count']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 17)

model training

In [121]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the model with default parameters
xgb_default = XGBRegressor(random_state=42)

# Fit the model to the training data
xgb_default.fit(X_train, y_train)

# Predict on the validation set
y_pred = xgb_default.predict(X_val)

# Compute RMSE
validation_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE (Default XGBRegressor):", validation_rmse)


Validation RMSE (Default XGBRegressor): 0.5369619867679355


In [126]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

# Define RMSE scorer
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Refined parameter grid with broader but reasonable ranges
param_grid = {
    'max_depth': [3, 4, 5],  # Broaden to include slightly deeper trees
    'min_child_weight': [1, 2, 3],  # Extend range for more flexibility
    'learning_rate': [0.01, 0.05, 0.1],  # Include a very low learning rate
    'n_estimators': [100, 200, 300],  # Slightly larger range for robustness
    'gamma': [0, 0.1, 0.2, 0.3],  # Include 0 for no regularization
    'subsample': [0.8, 0.9, 1.0],  # More choices for sampling
    'colsample_bytree': [0.8, 0.9, 1.0],  # More choices for column sampling
    'alpha': [0, 0.1, 0.3],  # L1 regularization
    'lambda': [1, 1.5, 2],  # L2 regularization
}

# Initialize the model
xgb = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    use_label_encoder=False,
    n_jobs=-1,
    random_state=42
)

# Randomized search with RMSE and more iterations to explore a larger space
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=50,  # Increase the number of iterations to explore more combinations
    cv=5,  # Use 5-fold cross-validation for better performance estimation
    scoring=rmse_scorer,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

# Best estimator
best_model = random_search.best_estimator_

# RMSE on validation set
y_pred = best_model.predict(X_val)
validation_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE:", validation_rmse)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'lambda': 1.5, 'gamma': 0.1, 'colsample_bytree': 0.9, 'alpha': 0}
Validation RMSE: 1.158725157756785


checking optuna

In [127]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.8 MB/s eta 0:00:00


In [128]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the objective function for Optuna
def objective(trial):
    # Define the search space for hyperparameters
    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Range for depth
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),  # Range for min_child_weight
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # Log-uniform distribution for learning rate
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),  # Number of trees
        'gamma': trial.suggest_float('gamma', 0, 1),  # Range for gamma (regularization)
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),  # Range for subsample
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),  # Range for colsample_bytree
        'alpha': trial.suggest_float('alpha', 0, 1),  # L1 regularization term
        'lambda': trial.suggest_float('lambda', 0, 1)  # L2 regularization term
    }

    # Initialize the model with these parameters
    model = XGBRegressor(
        objective='reg:squarederror',
        eval_metric='rmse',
        use_label_encoder=False,
        **param_grid,
        n_jobs=-1,
        random_state=42
    )

    # Fit the model and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    # Calculate RMSE for validation set
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    return rmse

# Create the Optuna study
study = optuna.create_study(direction='minimize')  # Minimizing RMSE
study.optimize(objective, n_trials=50, timeout=600)  # Limit to 50 trials or 10 minutes

# Print the best parameters and RMSE
print("Best Parameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Optionally, visualize the optimization process
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)


[I 2024-12-04 16:35:24,550] A new study created in memory with name: no-name-82c0eb97-96b9-4ce9-b652-c7327dea2318
[I 2024-12-04 16:35:59,103] Trial 0 finished with value: 0.5174321960415398 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'learning_rate': 0.08823660529557009, 'n_estimators': 409, 'gamma': 0.40866681425321805, 'subsample': 0.9669085073861472, 'colsample_bytree': 0.7075767699437705, 'alpha': 0.8813850047424574, 'lambda': 0.6901289961262688}. Best is trial 0 with value: 0.5174321960415398.
[I 2024-12-04 16:36:25,098] Trial 1 finished with value: 0.5229848554592209 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'learning_rate': 0.05406787323942546, 'n_estimators': 669, 'gamma': 0.17358578393835933, 'subsample': 0.9930936772712603, 'colsample_bytree': 0.946995753028099, 'alpha': 0.04509415478191947, 'lambda': 0.02249055243572806}. Best is trial 0 with value: 0.5174321960415398.
[I 2024-12-04 16:37:24,954] Trial 2 finished with value: 0.5172754888604778 and

Best Parameters: {'max_depth': 10, 'min_child_weight': 5, 'learning_rate': 0.017962744721141434, 'n_estimators': 712, 'gamma': 0.74183165927942, 'subsample': 0.7063983930090983, 'colsample_bytree': 0.7307654957712623, 'alpha': 0.3710483686967372, 'lambda': 0.8956182701348135}
Best RMSE: 0.5172754888604778


In [129]:
# Train the final model with the best parameters found by Optuna
best_params = study.best_params  # Get the best parameters from the study

final_model = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    use_label_encoder=False,
    **best_params,
    n_jobs=-1,
    random_state=42
)

# Train the model on the entire training data (X_train, y_train)
final_model.fit(X_train, y_train)


XGBRegressor(alpha=0.3710483686967372, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7307654957712623, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0.74183165927942,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.8956182701348135,
             learning_rate=0.017962744721141434, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=712, n_jobs=-1, ...)

In [122]:
test_path='data/final_test.parquet'
test_data = pd.read_parquet(Path(test_path))
print(test_data.columns)
test_data = external_data.example_estimator._merge_external_data(test_data)
test_data = external_data.example_estimator._encode_dates(test_data)

df=test_data
# Feature Engineering Example
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['day_period'] = pd.cut(df['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'])
df['installation_age_days'] = (pd.to_datetime(df['date']) - pd.to_datetime(df['counter_installation_date'])).dt.days

# Dropping Less Relevant Features
df = df.drop(columns=['counter_name', 'site_name', 'week_of_month'])
from geopy.distance import distance

# Coordinates of Paris city center
city_center = (48.8566, 2.3522)

# Add a new column 'distance_to_center' to the DataFrame
df['distance_to_center'] = df.apply(
    lambda row: distance(city_center, (row['latitude'], row['longitude'])).km,
    axis=1
)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
df['location_type'] = kmeans.fit_predict(df[['latitude', 'longitude']])
df['is_rainy'] = (df['rr1'] > 0).astype(int)
df['is_windy'] = (df['ff'] > 8).astype(int)
df['low_visibility'] = (df['vv'] < 1000).astype(int)
df['total_precipitation_24h'] = df[['rr1', 'rr3', 'rr6', 'rr12', 'rr24']].sum(axis=1)

df['temperature_c'] = df['t'] - 273.15  # Kelvin to Celsius conversion
df['is_hot'] = (df['temperature_c'] > 20).astype(int)  # Indicates hot weather
df['is_cold'] = (df['temperature_c'] < 5).astype(int)   # Indicates cold weather
df['is_snowy'] = (df['temperature_c'] < 0).astype(int)
df['bad_weather'] = ((df['is_rainy'] == 1) | (df['is_snowy'] == 1) | (df['low_visibility'] == 1)).astype(int)



# Exclude 'bike_count' and 'log_bike_count' from the columns of train_data_cleaned
columns_to_keep = selected_features

# Filter test_data to include only these columns
test_data = df[columns_to_keep]
test_data_cleaned = handle_missing_and_infs(test_data)


# Load the label encoders from the saved pickle file
with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

# Apply the label encoders to the test data
test_data_cleaned = test_data_cleaned.copy()

# Iterate through each categorical column in the test data
for col in test_data_cleaned.columns:
    if col in label_encoders:  # Check if the column was part of the training data encoding
        # Check if the column contains any missing values and handle them (if necessary)
        test_data_cleaned[col] = test_data_cleaned[col].astype(str)
        test_data_cleaned[col] = label_encoders[col].transform(test_data_cleaned[col])

# Check the updated test data
print("Updated test data:")
print(test_data_cleaned.head())


# # Identify numeric columns in the test data
# numeric_cols_test = test_data_cleaned.select_dtypes(include=['float64', 'int64']).columns

# # Initialize the scaler (using the one fitted on the training data)
# scaled_test_data = scaler.transform(test_data_cleaned[columns_to_scale])

# # Create a new DataFrame with the scaled numeric columns for the test data
# df_test_scaled = pd.DataFrame(scaled_test_data, columns=columns_to_scale)

# # Check the scaled test data
# print("Scaled test data:")
# print(df_test_scaled.head())
# # Transform the scaled test data to the selected components using the fitted PCA model
# df_pca_test = pd.DataFrame(pca_model.transform(df_test_scaled), columns=pca_columns)

# # Print the resulting DataFrame with the first few rows of PCA components for the test data
# print("DataFrame with 10 PCA components for test data:")
# print(df_pca_test.head())



Index(['counter_id', 'counter_name', 'site_id', 'site_name', 'date',
       'counter_installation_date', 'coordinates', 'counter_technical_id',
       'latitude', 'longitude'],
      dtype='object')
Updated test data:
      year  month  day  weekday  hour  holiday  workday  season  is_weekend  \
0     2021      9   10        4     1        0        1       3           0   
719   2021      9   10        4    13        0        1       3           0   
925   2021      9   10        4    17        0        1       3           0   
1012  2021      9   10        4    19        0        1       3           0   
1184  2021      9   10        4    22        0        1       3           0   

      day_period  installation_age_days   latitude  longitude  coordinates  \
0              3                   3157  48.846028   2.375429           10   
719            0                   3157  48.846028   2.375429           10   
925            0                   3157  48.846028   2.375429           1

In [123]:
# Ensure the test data aligns with the model's expectations
assert all(col in test_data_cleaned.columns for col in X_train.columns), "Test data must have the same columns as the training data."

# Predict using the trivial XGBRegressor
predictions = xgb_default.predict(test_data_cleaned)

# Display predictions
print("Predictions on test data:")



Predictions on test data:


In [131]:
def prepare_submission(predictions):
        """
        Save predictions
        """

        submission_df = pd.DataFrame({'id': range(len(predictions)), 'log_bike_count': predictions})
        submission_df.to_csv('seasoncsvoptuna.csv', index=False)
        print("Submission created.")

In [125]:
prepare_submission(predictions)

Submission created.


predicting using optuna

In [130]:
# Ensure the test data aligns with the model's expectations
assert all(col in test_data_cleaned.columns for col in X_train.columns), "Test data must have the same columns as the training data."

# Predict using the trivial XGBRegressor
predictions = final_model.predict(test_data_cleaned)

# Display predictions
print("Predictions on test data")

Predictions on test data:


In [132]:
prepare_submission(predictions)

Submission created.
